# Viscoelasticity
<strong>J. M. Maas @ IfB, ETH Zürich, v. 2026-01-19</strong>

<div class='alert alert-info'>
    Select ▸▸ <strong>Restart the kernel and run all cells</strong> to visualize the creep curves and run a widget for the TMSP.
</div>

In [ ]:
# Required libraries
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib as mpl
import ipywidgets as widgets
%matplotlib widget
import seaborn as sns

from pathlib import Path
from datetime import datetime

# Required modules
import utils.rheonomic_utils as rhutils

## Data import
---
Import the fitted data curves, parameters, and the scleronomic tests' elastic data.

In [ ]:
# Data paths
input_folder = Path('data/04_viscoelasticity')
figures_folder = Path('figures/04_viscoelasticity')

In [ ]:
# Collect files
files_params = list( input_folder.glob('ve*_kv_fit_parameters.csv') )
files_curves = list( input_folder.glob('ve*_kv_fit_strains.csv') )
file_meta = input_folder / 'meta_information.ods'
file_elasticity = input_folder / 'elasticity.xlsx'

# Import creep strains
rhs_curves = [float(file.stem.split('_')[0].strip('ve')) for file in files_curves]
df_curves = pd.concat([pd.read_csv(file, index_col=['sample', 'component', 'times'], parse_dates=[2]) for file in files_curves], keys=rhs_curves, names=['rh'])
df_curves.sort_index(inplace=True)

# Import creep parameters
rhs_params = [float(file.stem.split('_')[0].strip('ve')) for file in files_params]
df_params = pd.concat([pd.read_csv(file, index_col=['sample', 'component']) for file in files_params], keys=rhs_params, names=['rh'])

# Format creep dataframe
str2list = lambda s: [float(a) for a in s.strip('[] ').split(' ') if a != '']
df_params['tau'] = df_params['tau'].apply(str2list)
df_params['g'] = df_params['g'].apply(str2list)
df_params = df_params.explode(['tau', 'g']).set_index('tau', append=True).sort_index()

# Import elasticity
def import_elasticity(sheet):
    df = pd.read_excel(file_elasticity, sheet_name=sheet, index_col=[0])
    df = rhutils.interpolate_elasticity(df, 90)
    return df
    
sheets_elasticity = ['compression', 'tension', 'shear', 'average']
df_elasticity = pd.concat([import_elasticity(sheet) for sheet in sheets_elasticity], keys=sheets_elasticity)

df_elasticity.loc['average', 'G_TR'] = df_elasticity.loc['average', 'G_RT'].to_numpy()
df_elasticity.loc['average', 'G_LR'] = df_elasticity.loc['average', 'G_RL'].to_numpy()
df_elasticity.loc['average', 'G_LT'] = df_elasticity.loc['average', 'G_TL'].to_numpy()

# Show data
display(df_curves)
display(df_params)
display(df_elasticity)

Import the moisture content of each experiment and match it to the correct samples.

In [ ]:
# Import moisture contents

# Define import function
def import_creep_moisture(file_meta, samples=None):
    '''
    Imports and processes moisture content data from a specified Excel file.

    This function reads a spreadsheet containing moisture content data, cleans the data by removing 
    unnecessary columns and replacing invalid entries, calculates mean and standard deviation values 
    for each experimental condition, and fills missing data using group averages based on relative humidity (RH).
    It also associates sample names with calculated moisture content values and computes summary statistics 
    (mean, standard deviation, minimum, and maximum) for moisture content grouped by RH.

    Parameters:
    file_meta (str): The file path to the Excel spreadsheet containing the data.
    samples (list): Samples to include in the averaging. Average over all samples if None (default).

    Returns:
    tuple: A tuple containing:
        - df_mc_samples (DataFrame): A DataFrame linking sample names to their moisture content means, 
          standard deviations, and RH values.
        - mc_avg (DataFrame): A DataFrame containing summary statistics (mean, std, min, max) of 
          moisture content for each RH group.
    '''
    # Read spreadsheet
    df_mc = pd.read_excel(file_meta, sheet_name='Masses_Twins', index_col=[0,1])
    
    # Cleanup table
    df_mc.dropna(how='all', inplace=True)
    df_mc.drop(columns=['Note'], inplace=True)
    df_mc.replace('#VALUE!', np.nan, inplace=True)
    df_mc.sort_index(inplace=True)
    df_mc = df_mc.loc['Moisture content [-]']
    
    # Add additional information
    df_mc['mean'], df_mc['std'] = df_mc.mean(axis=1), df_mc.std(axis=1)  # do not set subsequentially for noting including mean in the std calculation
    df_mc['rh'] = [int(experiment.split('_')[1].strip('RH')) for experiment in df_mc.index]
    
    # Fill missing values
    df_mc_mean = df_mc[['mean', 'rh']].groupby('rh').mean()
    df_mc_std = df_mc[['mean', 'rh']].groupby('rh').std()
    na_idx = df_mc['mean'].isna()
    df_mc.loc[na_idx, 'mean'] = df_mc_mean.loc[df_mc.loc[na_idx, 'rh'], 'mean'].to_numpy()
    df_mc.loc[na_idx, 'std'] = df_mc_std.loc[df_mc.loc[na_idx, 'rh'], 'mean'].to_numpy()
    
    # Import sample names
    df_meta = rhutils.read_meta(file_meta, ['MACRO_' + name for name in df_mc.index.str.upper()])
    
    # Link sample names to moisture content
    df_mc_samples = pd.DataFrame(data={'sample': df_meta['name'], 'experiment': df_meta.index.get_level_values('experiment').str.strip('MACRO_')})
    df_mc_samples = pd.DataFrame(data={'sample': df_meta['name'].to_numpy(), 'experiment': df_meta.index.get_level_values('experiment').str.strip('MACRO_').str.replace('VISCO', 'Visco')})
    df_mc_samples['mc_mean'] = df_mc.loc[df_mc_samples['experiment'], 'mean'].to_numpy()
    df_mc_samples['mc_std'] = df_mc.loc[df_mc_samples['experiment'], 'std'].to_numpy()
    df_mc_samples['rh'] = df_mc.loc[df_mc_samples['experiment'], 'rh'].to_numpy()
    df_mc_samples.set_index('sample', inplace=True)
    
    # Calculate mean moisture content of valid samples
    if samples is None:
        samples = df_mc_samples.index.get_level_values('sample').unique()
    mc_avg = df_mc_samples.loc[ samples, ['mc_mean', 'rh'] ].groupby('rh')['mc_mean'].agg(['mean', 'std', 'min', 'max']) * 100

    return df_mc_samples, mc_avg


# Call import function
df_mc_samples, mc_avg = import_creep_moisture(
    file_meta=file_meta,
    samples=df_params.index.get_level_values('sample').unique()
)

# Show data
display(df_mc_samples)
display(mc_avg)

## Strain visualization
---
Visualize the strains and their corresponding Kelvin-Voigt series.

The Kelvin-Voigt series can be described as follows:
\begin{align}
C_{ij}^{-1} (t) = C_{0,ij}^{-1} + C_{\mathrm{ve},ij}^{-1} (t) = C_{0,ij}^{-1} \left( 1 + \sum_{k=1}^{N} g_k^\mathrm{ve} \left( 1 - \mathrm{e}^{-t/\tau_k} \right) \right) \text{ with } C_{0,ij}^{-1} = \frac{\epsilon_{0,ij}}{\sigma_{ij}} .
\end{align}

In [ ]:
# Reference curves
refs_creep = dict()  # t in min

# Curves from Hayashi et al. (1993)
refs_creep['E_tR'] = { 'tmax': 8367, 'fun': lambda t: 4.60e-4 * (1 + (t/1.8e5)**0.273) }
refs_creep['E_tL'] = { 'tmax': 5535, 'fun': lambda t: 5.92e-5 * (1 + (t/9.0e7)**0.242) }
refs_creep['G_RL'] = { 'tmax': 6800, 'fun': lambda t: 7.43e-4 * (1 + (t/5.9e5)**0.272) }

# Curves from Bengtsson et al. (2024)  # only tested over 24hrs and thus constant fit for larger times
# refs_creep['G_RT'] = { 'tmax': 1440, 'fun': lambda t: 0.0199 + 0.01382*(1 - np.exp(-t/(1.333*24))) }
# refs_creep['G_RL'] = { 'tmax': 1440, 'fun': lambda t: 1.434e-3 + 4.556e-5*(1 - np.exp(-t/(9.94*24))) }
# refs_creep['G_TL'] = { 'tmax': 1440, 'fun': lambda t: 1.612e-3 + 8.6523e-5*(1 - np.exp(-t/(8.87*24))) }

In [ ]:
# Function definitions
df_export_creep_strain = [pd.DataFrame()]
df_export_creep_fit = [pd.DataFrame()]

# Define strain plotting function
def plot_strains(sample_type, component, ax, dfs=df_curves, dfp=df_params, dfe=df_elasticity, mclim=[0.07, 0.20],
                 label=None, plot_exp_kw=dict(lw=0.6), plot_model_kw=dict(lw=1.2)):
    
    # Initialize formatting
    times_max = 30  # in days
    mcmin, mcmax = mclim
    color = lambda mc: sns.color_palette('crest_r', as_cmap=True)(1-(mc - mcmin)/(mcmax - mcmin))
    # color = lambda mc: 'black'
    def get_ls(mc):
        if mc < 0.1:
            return ':'
        elif mc < 0.15:
            return '--'
        else:
            return '-'
    
    # Select elasticity table
    def stype2index(sample_type):
        if 'c' in sample_type:
            return 'compression'
        elif 't' in sample_type:
            return 'tension'
        elif 's' in sample_type:
            return 'shear'
        else:
            raise ValueError(f'Invalid sample type {sample_type}.')

    # Define moisture function
    rh2moisture = lambda rhs: [rhutils.desorption_rh2mc(rh/100) if rh <= 65 else rhutils.adsorption_rh2mc(rh/100) for rh in rhs]

    # Get relative compliance ratios
    dfrc = get_elasticity_ratios(1/dfe, 65.0)  # reference relative humidity
    
    # Select data
    dfs = dfs.loc[ [sample_type in s for s in dfs.index.get_level_values('sample')] ].query(f'component == "{component}"')
    dfp = dfp.loc[ [sample_type in s for s in dfp.index.get_level_values('sample')] ].query(f'component == "{component}"')
    dfe = dfe.loc[stype2index(sample_type)]
    dfrc = dfrc.loc[stype2index(sample_type)]

    # Iterate over data
    samples = dfs.index.get_level_values('sample').unique()
    for sample in samples:

        # Get sample meta information
        data_model = dfp.loc[(slice(None), sample, slice(None))]
        eps0 = np.abs( data_model['eps0'].iloc[0] )
        stress = np.abs( data_model['stress'].iloc[0] )
        rh = data_model.index.get_level_values('rh')[0]
        # mc = rh2moisture([rh])[0]
        mc = df_mc_samples.loc[sample, 'mc_mean']
        stiffness_scleronomic = np.abs( rhutils.get_stiffness(dfe, sample, component, rh) )
        relative_compliance_scleronomic = np.abs( rhutils.get_stiffness(dfrc, sample, component, rh) )
        normalize_compliance = lambda compliance: (compliance - 1)  # * eps0 / stress * stiffness_scleronomic # * relative_compliance_scleronomic
        
        # Plot experimental data
        data_exp = dfs.loc[(slice(None), sample, slice(None))]
        datetimes = data_exp.index.get_level_values('times')
        times = (datetimes - datetimes[0]).total_seconds() / (3600*24)
        compliance_exp = normalize_compliance(data_exp['strain_corrected'])
        valid = ( np.abs(np.diff(compliance_exp, prepend=-1)) > 0 ) & ( times <= times_max )
        ax.plot(times[valid], compliance_exp[valid], c=color(mc), **plot_exp_kw)

        # Plot KV fitting data
        compliance_model = normalize_compliance(data_exp['KV_model'])
        ax.plot(times[valid], compliance_model[valid], c=color(mc), **plot_model_kw)

    # Add legend
    if not label is None:
        label_color = 'cornflowerblue' if sample_type in ['cR', 'cT', 'cLx', 'sRT', 'sRL', 'sTL'] else 'chocolate'
        text = ax.text(0.05, 0.955, label, transform=ax.transAxes, color=label_color, va='top', ha='left', fontsize='medium')
        text.set_bbox(dict(facecolor='white', alpha=0.6, edgecolor='gray', boxstyle='round, pad=0.15, rounding_size=0.2'))

    # Add R2
    R2 = dfp['R2'].mean()
    ax.text(0.98, 0.98, fr'$\bar{{R}}^2 = {R2:.2f}$', transform=ax.transAxes, color='black', va='top', ha='right', fontsize='small')

    # Format plot
    ax.set_xlim([0, times_max])
    ax.grid(True, which='major')
    ax.grid(True, which='minor', ls='-')

    df_export_creep_strain[0] = dfs
    df_export_creep_fit[0] = dfp


# Function for adding grid titles
def set_grid_title(gs, pos, text, pad=6.0, **kwargs):
    '''
    Creates a title above a selected grid space of a gridspec.
    Args:
        gs       (mpl.gridspec.GridSpec) : Matplotlib gridspec where to take the grid quadrant from.
        pos      (int, int)              : Vertical and horizontal quadrant index (y, x) as tuple.
        text     (str)                   : Title label.
        pad      (float)                 : Vertical padding between quadrant and text.
        **kwargs (Text properties)       : Keywords passed to the text formatting.
    '''
    # Default values
    fontsize = kwargs.get('fontsize', 12)
    va = kwargs.get('va', 'bottom')
    ha = kwargs.get('ha', 'center')
    
    # Collect gridspec information
    y, x = pos
    fig = gs.figure
    geometry = gs.get_geometry()
    gpos = gs.get_grid_positions(fig=fig)  # (bottom, top, left, right)

    # Check for input error
    if x >= geometry[1] or y >= geometry[0]:
        raise ValueError(f'Invalid grid position ({(y,x)}) for grid of geometry {geometry}.')

    # Transform coordinates
    transform = fig.transFigure.inverted()
    pad = transform.transform((0, pad))

    # Select position
    xcoord = (gpos[2][x] + gpos[3][x]) / 2
    ycoord = gpos[1][y] + pad[1]

    # Create text
    fig.text(xcoord, ycoord, text, ha=ha, va=va, fontsize=fontsize, **kwargs)
    return None


# Function for getting relative elasticity ratios
def get_elasticity_ratios(dfe, ref_rh):
    '''
    Return the ratios of elasticity in comparison to a specified reference relative humidity.
    Input:
        dfe     (pd.DataFrame) : Elasticity dataframe.
        rel_rh  (float)        : Reference relative humidity; must be in index of dfe.
    Output:
        dfe_rel (pd.DataFrame) : Elasticity dataframe with relative ratios to reference relative humidity.
    '''
    # Initialize
    sheets = dfe.index.get_level_values(0).unique()
    columns = dfe.columns
    dfe_rel = dfe.copy()

    # Iterate over all sheets and columns
    for sheet in sheets:
        for column in columns:
            # Calculate and store the relative values
            ref_value = dfe.loc[(sheet, ref_rh), column]
            dfe_rel.loc[sheet, column] = np.array( dfe.loc[sheet, column] / ref_value )

    # Return result
    return dfe_rel

The next cell creates a widgets for exploring and exporting the individual strain data and their fits.

In [ ]:
# Visualization function
def visualize_creep(component, ax):
    # Initialize
    ax.cla()
    component2label = {
        'E_cRR': (r'$E_{c,R}^{-1}$'),
        'E_tRR': (r'$E_{t,R}^{-1}$'),
        'E_cTT': (r'$E_{c,T}^{-1}$'),
        'E_tTT': (r'$E_{t,T}^{-1}$'),
        'E_cLL': (r'$E_{c,L}^{-1}$'),
        'E_tLL': (r'$E_{t,L}^{-1}$'),
        'G_RT': (r'$G_{RT}^{-1}$'),
        'G_TR': (r'$G_{TR}^{-1}$'),
        'G_RL': (r'$G_{RL}^{-1}$'),
        'G_LR': (r'$G_{LR}^{-1}$'),
        'G_TL': (r'$G_{TL}^{-1}$'),
        'G_LT': (r'$G_{LT}^{-1}$'),
        'E_cRL': (r'$\dfrac{ \nu_{c, RL} }{ E_{c,R} }$'),
        'E_tRL': (r'$\dfrac{ \nu_{t, RL} }{ E_{t,R} }$'),
        'E_cTR': (r'$\dfrac{ \nu_{c, TR} }{ E_{c,T} }$'),
        'E_tTR': (r'$\dfrac{ \nu_{c, TR} }{ E_{c,T} }$'),
        'E_cLT': (r'$\dfrac{ \nu_{c, LT} }{ E_{c,L} }$'),
        'E_tLT': (r'$\dfrac{ \nu_{t, LT} }{ E_{t,L} }$'),
    }
    
    # Get input
    constant = component.strip('KEG_')
    cparts = component.split('_')
    if 'E_' in component and (cparts[1][-2] != cparts[1][-1]):
        direction = 'exx'
        constant = constant[:-1]
    elif 'G_' in component:
        direction = 'exy'
        constant = 's' + constant
    else:
        direction = 'eyy'
        constant = constant[:-1]
    if constant in ['cL', 'tL']:
        constant += 'x'

    # Create plot
    plot_strains(constant, direction, ax, mclim=mclim, label=component2label[component], plot_exp_kw=plot_exp_kw, plot_model_kw=plot_model_kw)

    # Format plot
    ax.set_xlabel('Time [d]')
    ax.set_ylabel(r'Creep compliance $C_{\mathrm{ve},ij}^{-1} (t) / C_{0,ij}^{-1}$ [-]')
    

# Export function
def export_csv_creep(button):
    if len(df_export_creep_fit) > 0 and len(df_export_creep_strain) > 0:
        export_path_fit = f'export_creep_fit_{w_creep_components.value}.csv'
        export_path_strain = f'export_creep_points_{w_creep_components.value}.csv'
        df_export_creep_fit[0].to_csv(export_path_fit)
        df_export_creep_strain[0].to_csv(export_path_strain)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with w_creep.children[-1]:
            w_creep.children[-1].clear_output()
            print(f'({timestamp}) Successfully saved data to {export_path_fit} and {export_path_strain}.')
button_creep = widgets.Button(description='Export CSVs', button_style='primary', icon='save')
button_creep.on_click(export_csv_creep)
display(button_creep)


# Initialize formatting
mclim = np.array([mc_avg['min'].min(), mc_avg['max'].max()]) / 100
plot_exp_kw = dict(lw=0.6, ls=':', alpha=1.0)
plot_model_kw = dict(lw=1.2, ls='-', alpha=0.8)

# Create widgets
fig, ax = plt.subplots()
components = ['E_cRR', 'E_tRR', 'E_cTT', 'E_tTT', 'E_cLL', 'G_RT', 'G_TR', 'G_RL', 'G_LR', 'G_TL', 'G_LT', 'E_cRL', 'E_tRL', 'E_cTR', 'E_tTR', 'E_cLT', 'E_tLT']
w_creep_components = widgets.Dropdown(options=components, description='component')
w_creep = widgets.interactive(
    visualize_creep,
    component=w_creep_components,
    ax=widgets.fixed(ax)
)
display(w_creep)

Return the number of sample curves per anatomical direction and climate.

In [ ]:
# Count number of curves per climate
df_numbers = df_params.query(f'tau == {df_params.index.get_level_values("tau").min()}').copy()
df_numbers['direction'] = [sample.split('_')[1].split('-')[0] for sample in df_numbers.index.get_level_values('sample')]
df_numbers = df_numbers.groupby(['rh', 'direction', 'component'])[['g']].count().sort_index()
df_numbers.rename(columns={'g': 'number'}, inplace=True)
df_numbers.query('direction != "cL" and direction != "tL"', inplace=True)

df_numbers.to_csv(input_folder / 've_numbers.csv')

## Coefficient visualization
---
Visualize the distribution of Kelvin-Voigt coefficients.

<strong style='color:red'>The Kelvin-Voigt coefficients in this plot got normalized to the scleronomic elastic compliance!</strong>

In [ ]:
# Prepare data

# Initialize functions
rh2moisture = lambda rhs: [rhutils.desorption_rh2mc(rh/100) if rh <= 65 else rhutils.adsorption_rh2mc(rh/100) for rh in rhs]
mcmin, mcmax = 6, 21
offset_fun = lambda mc: 0.55 * ((mc - mcmin) / (mcmax - mcmin) - 0.5)

def stype2index(sample_type):
    if 'c' in sample_type:
        return 'compression'
    elif 't' in sample_type:
        return 'tension'
    elif 's' in sample_type:
        return 'shear'
    else:
        raise ValueError(f'Invalid sample type {sample_type}.')
        
def get_stiffness(df):
    indexes = df.index
    stiffnesses = []
    for index in indexes:
        stype = df.loc[index, 'type']
        rh, sample, component, tau = index
        stiffness = np.abs( rhutils.get_stiffness(df_elasticity.loc[stype2index(stype)], sample, component, rh) )
        stiffnesses.append(stiffness)
    return stiffnesses

# Create DataFrame
df = df_params.copy()
df['type'] = [sample.split('_')[1].split('-')[0] + f'_{component}' for sample, component in zip(df.index.get_level_values('sample'), df.index.get_level_values('component'))]
df['C_scleronomic'] = get_stiffness(df)
df['tau'] = df_params.index.get_level_values('tau')
df['g_abs'] = df_params['g'] # * np.abs( df_params['eps0'] / df_params['stress'] ) * df['C_scleronomic']
# df.loc[(slice(None), slice(None), 'exx', slice(None)), 'g_abs'] = df.loc[(slice(None), slice(None), 'exx', slice(None)), 'g']
# df.loc[(slice(None), slice(None), 'eyy', slice(None)), 'g_abs'] = df.loc[(slice(None), slice(None), 'eyy', slice(None)), 'g']
df['mc'] = np.array( rh2moisture(df_params.index.get_level_values('rh')) ) * 100

In [ ]:
# Initialize plot
margin = [0.06, 0.02, 0.10, 0.07]  # [left, right, bottom, top]
fig, axs_all = plt.subplots(4, 2, figsize=(10, 6), width_ratios=[0.66, 0.34], gridspec_kw=dict(hspace=0, wspace=0, left=margin[0], right=1-margin[1], bottom=margin[2], top=1-margin[3]))
axs, axsr = axs_all[:, 0], axs_all[:, 1]

# Format left axes
axs_top = np.array( [ax.twiny() for ax in axs] )
[ax.tick_params(labelbottom=False) for ax in axs[:-1]]
[ax.tick_params(labeltop=False) for ax in axs_top[1:]]

# Format right axes
axsr_top = np.array( [ax.twiny() for ax in axsr] )
[ax.yaxis.set_label_position('right') for ax in axsr]
[ax.yaxis.tick_right() for ax in axsr]
[ax.tick_params(labelbottom=False) for ax in axsr[:-1]]
[ax.tick_params(labeltop=False) for ax in axsr_top[1:]]


# Plot settings
fcs = ['lightsteelblue', 'sandybrown']
ecs = ['cornflowerblue', 'chocolate']
labels = [r'compression, $1/G_{RT}$, $1/G_{RL}$, $1/G_{TL}$',
          r'tension, $1/G_{TR}$, $1/G_{LR}$, $1/G_{LT}$']
alpha = [0.6, 0.5]
box_width = 0.18
ticks_fontsize = 'medium'


# Iterate over taus
taus = df['tau'].unique()
for i, tau in enumerate(taus):
    # Plot compression RTL data
    types = ['cR_eyy', 'cT_eyy', 'cLx_eyy', 'sRT_exy', 'sRL_exy', 'sTL_exy']
    data = df.query(f'type in {types} and tau == {tau}')
    rhutils.boxplot(data, x='type', y='g_abs', hue='mc', order=types, legend=False, ax=axs[i], fc=fcs[0], ec=ecs[0], marker='o', box_width=box_width, lw=1.2, alpha=alpha[0], offset_fun=offset_fun)
    
    # Plot tension LTR data
    types = ['tR_eyy', 'tT_eyy', 'tLx_eyy', 'sTR_exy', 'sLR_exy', 'sLT_exy']
    data = df.query(f'type in {types} and tau == {tau}')
    rhutils.boxplot(data, x='type', y='g_abs', hue='mc', order=types, legend=False, ax=axs_top[i], fc=fcs[1], ec=ecs[1], marker='o', box_width=box_width, lw=1.2, alpha=alpha[1], offset_fun=offset_fun)

    # Plot lateral compression data
    types = ['cT_exx', 'cR_exx', 'cLx_exx']
    data = df.query(f'type in {types} and tau == {tau}')
    rhutils.boxplot(data, x='type', y='g_abs', hue='mc', order=types, legend=False, ax=axsr[i], fc=fcs[0], ec=ecs[0], marker='o', box_width=box_width, lw=1.2, alpha=alpha[0], offset_fun=offset_fun)
    
    # Plot lateral tension data
    types = ['tT_exx', 'tR_exx', 'tLx_exx']
    data = df.query(f'type in {types} and tau == {tau}')
    rhutils.boxplot(data, x='type', y='g_abs', hue='mc', order=types, legend=False, ax=axsr_top[i], fc=fcs[1], ec=ecs[1], marker='o', box_width=box_width, lw=1.2, alpha=alpha[1], offset_fun=offset_fun)


# Add taus
tau_xl, tau_xr, tau_y = 0.985, 0.97, 0.92
tau_labels = [r'$\tau_{1,ij} = 2\,\mathrm{h}$', r'$\tau_{2,ij} = 20\,\mathrm{h}$', r'$\tau_{3,ij} = 200\,\mathrm{h}$', r'$\tau_{4,ij} = 2000\,\mathrm{h}$']
tau_label_format = dict(va='top', ha='right', size='medium', weight='bold')
bbox = dict(facecolor='white', alpha=0.6, edgecolor='gray', boxstyle='round, pad=0.15, rounding_size=0.2')
for label, axl, axr in zip(tau_labels, axs, axsr):
    textl = axl.text(tau_xl, tau_y, label, transform=axl.transAxes, **tau_label_format)
    textr = axr.text(tau_xr, tau_y, label, transform=axr.transAxes, **tau_label_format)
    textl.set_bbox(bbox)
    textr.set_bbox(bbox)


# Format left labels
axs[0].text(0.02, 0.92, '(a)', transform=axs[0].transAxes, va='top', ha='left', weight='bold', size='x-large', family='arial')
axs[-1].set_xticklabels([r'$E_{c,R}^{-1}$', r'$E_{c,T}^{-1}$', r'$E_{c,L}^{-1}$', r'$G_{RT}^{-1}$', r'$G_{RL}^{-1}$', r'$G_{TL}^{-1}$'], color=ecs[0], fontsize=ticks_fontsize)
axs_top[0].set_xticklabels([r'$E_{t,R}^{-1}$', r'$E_{t,T}^{-1}$', r'$E_{t,L}^{-1}$', r'$G_{TR}^{-1}$', r'$G_{LR}^{-1}$', r'$G_{LT}^{-1}$'], color=ecs[1], fontsize=ticks_fontsize)
def float2str(num):
    exponent = np.ceil( np.log10( num ) ) - 1
    base = num / 10**exponent
    return f'{base:.1f} \cdot 10^{int(exponent)}'
# [ax.set_ylabel(rf'$\tau_{i+1} = {int(tau/(3600))}$ h') for i, (ax, tau) in enumerate(zip(axs, taus))]
# [ax.yaxis.set_label_position('right') for ax in axs]

# Format right labels
axsr[0].text(0.035, 0.92, '(b)', transform=axsr[0].transAxes, va='top', ha='left', weight='bold', size='x-large', family='arial')
axsr[-1].set_xticklabels([r'$\dfrac{\nu_{c,TR}}{E_{c,T}}$', r'$\dfrac{\nu_{c,RL}}{E_{c,R}}$', r'$\dfrac{\nu_{c,LT}}{E_{c,L}}$'], color=ecs[0], fontsize=ticks_fontsize)
axsr_top[0].set_xticklabels([r'$\dfrac{\nu_{t,TR}}{E_{t,T}}$', r'$\dfrac{\nu_{t,RL}}{E_{t,R}}$', r'$\dfrac{\nu_{t,LT}}{E_{t,L}}$'], color=ecs[1], fontsize=ticks_fontsize)

# Format figure labels
fig.supxlabel('Compliance component', x=(1-margin[1]+margin[0])/2, y=0.008)
fig.supylabel(r'Kelvin-Voigt coefficients $g^\mathrm{ve}_{k,ij}$ [-]', x=0.005, y=(1-margin[3]+margin[2])/2)

# Format left grid
ylims = [[0, 2.8], [0, 2.8], [0, 2.8], [0, 2.8]]
steps = [0.2, 0.2, 0.2, 0.2]
[ax.set_ylim(ylim) for ax, ylim in zip(axs, ylims)]
axs[0].set_yticks(np.arange(0, 2.8, 1.0))
axs[1].set_yticks(np.arange(0, 2.8, 1.0))
axs[2].set_yticks(np.arange(0, 2.8, 1.0))
axs[3].set_yticks(np.arange(0, 2.8, 1.0))
[ax.set_yticks(np.arange(ylim[0]-ylim[0]%step, ylim[1]+1e-8, step), minor=True) for ax, ylim, step in zip(axs, ylims, steps)]
[ax.grid(True, which='major') for ax in axs]
[ax.grid(True, which='minor', ls=':', lw=0.5, color='lightgray') for ax in axs]

# Format right grid
ylimsr = [[0, 5.6], [0, 5.6], [0, 5.6], [0, 11.2]]
stepsr = [0.4, 0.4, 0.4, 0.8]
[ax.set_ylim(ylim) for ax, ylim in zip(axsr, ylimsr)]
axsr[0].set_yticks(np.arange(0, 5.8, 2.0))
axsr[1].set_yticks(np.arange(0, 5.8, 2.0))
axsr[2].set_yticks(np.arange(0, 5.8, 2.0))
axsr[3].set_yticks(np.arange(0, 12.0, 4.0))
[ax.set_yticks(np.arange(ylim[0]-ylim[0]%step, ylim[1]+1e-8, step), minor=True) for ax, ylim, step in zip(axsr, ylimsr, stepsr)]
[ax.grid(True, which='major') for ax in axsr]
[ax.grid(True, which='minor', ls=':', lw=0.5, color='lightgray') for ax in axsr]


# Add inset axis
inset_base = [6, 1.0] # [x, y]
inset_ax = axs_top[0]
inset_color = 'black'
inset_lw = 0.8
inset_tickheight = 0.15
inset_fontsize = 'medium'
inset_ax.arrow(inset_base[0]+offset_fun(mcmin), inset_base[1], offset_fun(mcmax)-offset_fun(mcmin), 0, color=inset_color, lw=inset_lw, head_width=0.1, head_length=0.1, overhang=0.4)
inset_ax.text(inset_base[0]+offset_fun(mcmax)+0.25, inset_base[1], r'$\omega$', ha='center', va='center', fontsize=inset_fontsize)
for i, mc in enumerate(df['mc'].unique()):
    # Add tick lines
    tick_x, tick_y = inset_base[0]+offset_fun(mc), inset_base[1]
    inset_ax.plot([tick_x, tick_x], [tick_y-inset_tickheight/2, tick_y+inset_tickheight/2], color=inset_color, label='__no_legend__', lw=inset_lw)
    # Add tick labels
    if i % 2 == 0: # even ticks
        inset_ax.text(tick_x, tick_y-inset_tickheight, f'{mc/100:.2f}', ha='center', va='top', color=inset_color, fontsize=inset_fontsize)
    else: # odd ticks
        inset_ax.text(tick_x, tick_y+inset_tickheight, f'{mc/100:.2f}', ha='center', va='bottom', color=inset_color, fontsize=inset_fontsize)


# Save figure
save_path = figures_folder / f'kv_coefficients.png'
fig.savefig(save_path, dpi=300)
print(f'Successfully saved figure under {save_path}.')

## Clustering
---
To obtain the minimum number of required experiments, the fitted creep curves are subsequently clustered into groups of similar creep behavior.

The following clustering algorithm clusters the viscoelastic creep after 30 days per symmetrized direction, where the creep at the different RH are the respective features. For that purpose, the subsequent cell constructs the feature array, where the rows resemble the individual directions and the columns the features, i.e., the creep after 30 days for the individual RH.

In [ ]:
# Collect creep coefficients
def get_avg_gi(constant, rh, symmetrize=False, df_p=df_params):
    '''
    Determine the average Kelvin-Voigt coefficients for a given constant.
    Input:
        constant   (str)          : Component of the creep compliance matrix.
                                    Options: E_cRR, E_tRR, E_cTT, E_tTT, E_cLL, E_tLL,
                                    G_RT, G_TR, G_RL, G_LR, G_TL, G_LT, E_cRL, E_tRL,
                                    E_cTR, E_tTR, E_cLT, E_tLT
        rh         (float)        : Relative humidity of group.
        symmetrize (bool)         : Form average between selected constant and
                                    complementary loading case.
        df_p       (pd.DataFrame) : Dataframe with Kelvin-Voigt parameters per sample.
    Output:
        g_avg    (pd.Series)    : Average Kelvin-Voigt coefficients, indexed by tau.
    Example:
        get_avg_gi('G_RT', 35)
    '''
    # Clean data
    df_p = df_p.loc[ [not ('_cL-' in sample or '_tL-' in sample) for sample in df_p.index.get_level_values('sample')] ]
    
    # Identify index
    parts = constant.split('_')
    match parts[0]:
        case 'E':
            loading_type = parts[1][0]
            direction = parts[1][1:]
            component = 'eyy' if direction[0] == direction[1] else 'exx'
            direction = direction[0].upper()
            sample_type = loading_type + direction
            sample_type_complementary = ('t' if loading_type == 'c' else 'c') + direction
        case 'G':
            loading_type = 's'
            direction = parts[1].upper()
            component = 'exy'
            sample_type = loading_type + direction
            sample_type_complementary = loading_type + direction[::-1]
        case _:
            raise ValueError(f'Invalid constant "{constant}".')
    
    # Collect data
    data = df_p.loc[rh].query(f'component == "{component}"')
    samples = [sample_type in sample for sample in data.index.get_level_values('sample')]
    samples_complementary = [sample_type_complementary in sample for sample in data.index.get_level_values('sample')]
    data = data.loc[ (np.any([samples, samples_complementary], axis=0) if symmetrize else samples) ]

    # Get average
    data_mean = data.groupby('tau')[['g', 'C_inv', 'R2']].mean()
    g_avg = data_mean['g']

    # Return result
    return g_avg


# Collect cluster features
def get_feature(constant, rhs, t=30):
    '''
    Collect cluster feature for given constant.
    Input:
        constant (str)        : Selected direction, which gets symmetrized with its
                                complementary loading case.
        rhs      (list)       : List of relative humidities, resembling the object features.
        t        (float)      : Time in days where creep compliance is evaluated.
    Output:
        features (np.ndarray) : Creep compliance after t days at given rhs.
    '''
    # Initialize
    t = np.array([t*24*60*60])  # transform time from days to seconds
    
    # Calculate creep after certain time for all climates
    features = []
    for rh in rhs:
        G = get_avg_gi(constant, rh, symmetrize=True)
        tau = G.index.to_numpy()
        g = G.values
        C_inv = np.sum( g*(1-np.exp(-np.array([t]).T/tau)), axis=1 )
        features += list(C_inv)

    # Return result
    return features


# Assemble features
groups = ['E_cRR', 'E_cTT', 'E_cLL', 'G_RT', 'G_RL', 'G_TL', 'E_cTR']  # ignore 'E_cRL' and 'E_cLT' due to their inaccuracy
rhs = sorted(df_params.index.get_level_values('rh').unique())
cluster_time = 30  # time to determine clusters at in [days]
features = [get_feature(group, rhs=rhs, t=cluster_time) for group in groups]  # columns = object features, rows = objects

Subsequently, form clusters from the selected features. Consider to omit $\nu_{RL} / E_R$ and $\nu_{LT} / E_L$ due to their low accuracy. The markers indicate the cluster values.

The next cell creates a widget to explore different numbers of clusters.

In [ ]:
# Create cluster exploration widget

# Determine clusters
def get_clusters(number_of_clusters, obs, whiten=False):
    # Structure data to be clustered
    if whiten:
        obs_w = sp.cluster.vq.whiten(features) # normalizes data to improve the result of kmeans
        obs_std = np.std(obs, axis=0)
    else:
        obs_w = obs
        obs_std = 1.0
    
    # Perform cluster analysis
    clust_pos, _ = sp.cluster.vq.kmeans(obs_w, number_of_clusters, iter=200, seed=0)
    
    # Read out the found clusters
    clust_index, clust_distance = sp.cluster.vq.vq(obs_w, clust_pos)
    clust_pos_original = clust_pos * obs_std # invert results back to non-normalized space

    # Get maximum distance of objects of same cluster
    dist_max = []
    obs_dist = []
    relative_distance = True
    for i in sorted(np.unique(clust_index)):
        objs = np.where(clust_index == i)[0]
        coords = np.array(obs)[objs]
        if relative_distance:
            dists = np.array( [np.abs(coords[j] - coords[k]) / (np.abs(coords[j] + coords[k])/2) for j in range(len(coords)) for k in range(j, len(coords)) if j != k] )
        else:
            dists = np.array( [np.abs(coords[j] - coords[k]) for j in range(len(coords)) for k in range(j, len(coords)) if j != k] )
        obs_dist.append(dists if len(dists) > 0 else np.zeros((1, np.shape(obs_w)[1])))
        
    obs_dist_mean = np.array( [np.mean(a) for a in obs_dist] )
    obs_dist_max = np.array( [np.max(a, axis=0) for a in obs_dist] )

    # Return result
    clusters = {
        'clust_index': clust_index,
        'clust_pos': clust_pos,
        'clust_pos_original': clust_pos_original,
        'clust_distance': clust_distance,
        'obs_mean_distance': obs_dist_mean,  # mean distance between the objects of the respective group
        'obs_max_distance': obs_dist_max  # maximum distance between the objects of the respective group
    }
    return clusters


# Visualization function
def visualize_clusters(number_of_clusters, axs):
    # Initialize
    [ax.cla() for ax in axs.flatten()]
    
    # Read out results
    cluster = get_clusters(number_of_clusters, features)
    clust_index, clust_pos_original, clust_distance = cluster['clust_index'], cluster['clust_pos_original'], cluster['clust_distance']
    obj_max_distance = cluster['obs_max_distance']
    
    around = lambda a: np.round(a, 3)
    clust_out = 'Cluster memberships:\n' + '\n'.join(
        [f' {group}: {clust_index[i]:d} (cluster value: {around(clust_pos_original[clust_index[i]])}, ' + \
         f'maximum distance between objects: {around(obj_max_distance[clust_index[i]])}, ' + \
         f'cluster distance: {clust_distance[i]:.3f})' for i, group in enumerate(groups)
        ])
    print(clust_out)
    print(f'Mean cluster distance: {np.mean(clust_distance):.3f}.')
    print(f'Maximum cluster distance: {np.max(clust_distance):.3f}.')
    
    # Plot cluster
    group_formats = {
        'E_cRR': dict(pos=[0, 0]),
        'E_cTT': dict(pos=[1, 0]),
        'E_cLL': dict(pos=[2, 0]),
        'G_RT': dict(pos=[0, 1]),
        'G_RL': dict(pos=[1, 1]),
        'G_TL': dict(pos=[2, 1]),
        'E_cTR': dict(pos=[0, 2]),
        'E_cRL': dict(pos=[1, 2]),
        'E_cLT': dict(pos=[2, 2]),
    }
    cmap = mpl.colormaps['tab10']
    rhs = sorted(df_params.index.get_level_values('rh').unique())
    
    for i, group in enumerate(groups):
        # Select formatting
        pos = group_formats[group]['pos']
        ax = axs[pos[0], pos[1]]
        cluster = clust_index[i]
        color = cmap(cluster)
        time_factor = 24*60*60  # transform days to seconds
    
        # Determine creep curves
        for j, rh in enumerate(rhs):
            G_i = get_avg_gi(group, rh, symmetrize=True)
            g = G_i.values
            tau = G_i.index.to_numpy()
            times = np.linspace(0, cluster_time*time_factor, 100)
            compliances = np.sum( g*(1-np.exp(-np.array([times]).T/tau)), axis=1 )
        
            # Plot curve
            label = rf'${group.replace("c", "").replace("t", "").replace("_", "_{")}}}$ (cluster {cluster})' if j == 0 else '__no_label__'
            ax.plot(times/time_factor, compliances, color=color, label=label)
    
        # Plot cluster result
        clust_result = clust_pos_original[cluster]
        ax.scatter(np.full(clust_result.shape, cluster_time), clust_result, s=30, marker='o', fc=(0, 0, 0, 0), color=color, zorder=10)
    
        # Format plot
        ax.legend(loc='upper left')
        ax.grid(True)


# Initialize figure
fig, axs = plt.subplots(3, 3, figsize=(9, 7), gridspec_kw=dict(hspace=0, wspace=0), sharex=True, sharey=True)
fig.suptitle('Clustering')
fig.supxlabel('Time [d]')
fig.supylabel('Normalize creep compliance [-]')
fig.tight_layout()

# Create widget
widgets.interactive(
    visualize_clusters,
    number_of_clusters=widgets.Dropdown(value=4, options=np.arange(0, 7, dtype=int)),
    axs=widgets.fixed(axs)
)

The next cell determines the increment of cluster distance over increasing number of clusters. For a minimum number of experiments, it would be advisable to keep the number of clusters as low as necessary. <i>err_mean</i> and <i>err_max</i> denote the mean and maximum relative difference between the creep compliances of a same group. <!--, and <i>i_err_max</i> denotes the cluster number where this maximum difference occurs.-->

In [ ]:
# Iterate over different numbers of clusters
numbers_of_clusters = list( range(1, len(groups)+1) )

clusters = [get_clusters(n, features) for n in numbers_of_clusters]
indexes = [cluster['clust_index'] for cluster in clusters]
distances = [cluster['clust_distance'] for cluster in clusters]
obj_mean_distances = [cluster['obs_mean_distance'] for cluster in clusters]
obj_max_distances = [cluster['obs_max_distance'] for cluster in clusters]

d_mean = [np.mean(d) for d in distances]
d_max = [np.max(d) for d in distances]
err_mean = [np.round( d, 3) for d in obj_mean_distances]
err_max = [np.round( np.max(d, axis=1), 3) for d in obj_max_distances]
err_mean_global = [np.mean(d) for d in obj_max_distances]
err_max_global = [np.max(d) for d in obj_max_distances]
# i_err_max = [np.unravel_index(np.argmax(d), d.shape)[0] for d in obj_max_distances]

# Assemble results
labels = [group.replace('c', '').replace('t', '') for group in groups]
dict_index = dict(zip(['i_'+label for label in labels], np.array(indexes).T))
dict_dist = dict(zip(['dist_'+label for label in labels], np.array(distances).T))
clust_df = pd.DataFrame(
    data=dict_index | dict_dist,
    index=pd.Index(numbers_of_clusters, name='cluster')
)
clust_df['dist_mean'] = d_mean
clust_df['dist_max'] = d_max
clust_df['delta_dist_mean'] = clust_df['dist_mean'].diff()
clust_df['delta_dist_max'] = clust_df['dist_max'].diff()
clust_df['err_mean'] = err_mean
clust_df['err_max'] = err_max
clust_df['err_mean_global'] = err_mean_global
clust_df['err_max_global'] = err_max_global
# clust_df['i_err_max'] = i_err_max

# Return results
display(clust_df)

## Time-moisture superposition
---
Visualize the results of the time-moisture superposition principle (TMSP).

In [ ]:
# Import data tables
tmsp_folder = input_folder / 'tmsp'

# Parameters
df_master = pd.read_csv(tmsp_folder / 'tmsp_master_curve_parameters.csv', index_col=['constant', 'element'])
df_time_shift = pd.read_csv(tmsp_folder / 'tmsp_time_shift_parameters.csv', index_col=['constant'])
df_ln_a = pd.read_csv(tmsp_folder / 'tmsp_time_shifts.csv', index_col=['constant', 'sample'])

# Sample information
df_meta = pd.read_csv(tmsp_folder / 'tmsp_meta.csv', index_col=['sample', 'component'])
df_creep = pd.read_csv(tmsp_folder / 'tmsp_sample_strains.csv', index_col=['sample', 'component', 'times'], parse_dates=['times'])

# Model response
df_models_master = pd.read_csv(tmsp_folder / 'tmsp_master_models.csv', index_col=['component', 'x'])
df_models_time_shift = pd.read_csv(tmsp_folder / 'tmsp_time_shift_models.csv', index_col=['component', 'x'])
df_log = pd.read_csv(tmsp_folder / 'tmsp_log.csv', index_col=['constant', 'iteration'])

In [ ]:
# Define visualization function
df_export_tmsp_master = [pd.DataFrame()]
df_export_tmsp_ln_a = [pd.DataFrame()]

# Initialize settings
kv_model = lambda t, gamma_0, g, tau: gamma_0 * ( 1 + np.sum( g*(1-np.exp(-np.array([t]).T/tau)), axis=1 ) )
settings = dict()
settings['mode'] = 'experimental'  # data source
settings['mc_ref'] = 12.5  # reference moisture content
settings['fun_ln_a'] = lambda x, p, exp_mc, exp_ln_a: p[1] + (np.max(exp_ln_a[np.where(exp_mc > 17)[0]]) - p[1]) / (1+np.exp(-1*(x-p[0])))  # logistic function
settings['fun_ln_a_extra_params'] = lambda exp_mc, exp_ln_a: {  # extra parameters to be stored in dataframe during fitting
    'y_r': np.max(exp_ln_a[np.where(exp_mc > 17)[0]]),
}
settings['fun_ln_a_p0'] = lambda exp_mc, exp_ln_a: [12.5, np.mean(exp_ln_a[np.where(exp_mc < 10)[0]])]
settings['fun_ln_a_bounds'] = lambda exp_mc, exp_ln_a: [ (10, 20), (np.min(exp_ln_a), np.max(exp_ln_a)) ]
settings['compliance_type'] = 'J_relative'

match settings['compliance_type']:  # adapt function kv_model(t, gamma_0, g, tau) to selected compliance type
    case 'J_total':
        settings['fun_master_curve'] = lambda t, tau, p: kv_model( t, p[0], p[1:], tau )
    case 'J_c':
        settings['fun_master_curve'] = lambda t, tau, p: kv_model( t, 1, p, tau ) - 1
    case 'J_relative':
        settings['fun_master_curve'] = lambda t, tau, p: kv_model( t, 1, p, tau ) - 1
    case _:
        raise ValueError(f"Invalid compliance type '{settings['compliance_type']}'.")


# Transform constant to samples
def extract_samples(df_meta, constant):
    # Collect matching sasmples
    data = df_meta.query(f"constant == '{constant}'")
    samples = sorted( data.index.get_level_values('sample').unique() )
    components = sorted( data.index.get_level_values('component').unique() )

    # Check found data
    if len(components) != 1:
        raise ValueError(f'Invalid number of components found. Expected one, but found {components}')
    component = components[0]

    # Return results
    return samples, component


# Collect strains
def collect_strains(sample, component, mode, df_creep):
    '''
    Collects the creep strains for a given samples and component.
    Sections with zero strain changes are removed from data.
    Input:
        sample    (str)        : Sample name to collect the strains for.
        component (str)        : Strain component, i.e., 'exx', 'exy', or 'eyy'.
        mode      (str)        : Collection mode, i.e., 'experimental', 'fitted', or 'average'.
    Output:
        times     (np.ndarray) : Times in seconds.
        strains   (np.ndarray) : Absolute creep strains.
    '''
    # Select time series with strain data
    df = df_creep.loc[(sample, component)]
    compliance = df_meta.loc[(sample, component), 'compliance']
    match mode:
        case 'experimental':
            times = df.index.get_level_values('times')
            times = (times - times[0]).total_seconds()
            strains = df['strain_experiment'].to_numpy() * compliance
        case 'fitted':
            times = df.index.get_level_values('times')
            times = (times - times[0]).total_seconds()
            strains = df['KV_model'].to_numpy() * compliance
        case 'average':
            # Calculate from average fitted parameters
            constant = df_meta.loc[(sample, component), 'constant']
            rh = df_meta.loc[(sample, component), 'rh']
            df = df_creep_avg.loc[(constant, rh)]
            compliance = 1 / df_meta.query(f'constant == "{constant}" and rh == {rh}')['stiffness'].mean()
            times = df.index.get_level_values('times')
            strains = df['strain'].to_numpy() * compliance
        case _:
            raise ValueError(f"Invalid mode '{mode}'.")
    # Crop constant strains
    non_zero_diff = np.where( ~np.isclose(np.diff(strains, prepend=0), 0) )[0]
    times = times[non_zero_diff]
    strains = strains[non_zero_diff]
    # Adapt to compliance type:
    match settings['compliance_type']:
        case 'J_total':
            pass
        case 'J_relative':
            strains = strains / strains[0] - 1
        case 'J_c':
            strains = strains - strains[0]  # not necessary according to Hajikarimi et al. (2018)
        case _:
            raise ValueError(f"Invalid compliance type '{settings['compliance_type']}'.")
    # Return result
    return times, strains


# Define visualization function
def visualize_tmsp_model(constant, moisture, show_report, show_confidence, df_master, df_time_shift, df_ln_a, df_log, df_models_master, df_models_time_shift, df_meta, df_creep, axs):
    # Initialize
    mode = settings['mode']
    model_color = 'red'
    confidence_color = 'gray'
    model_lw = 2.5
    sigma = 1  # standard deviation multiplier for confidence bands and prediction bands
    [ax.cla() for ax in axs]
    mcmin, mcmax = df_meta['mc'].min(), df_meta['mc'].max()
    mc2color = lambda mc: sns.color_palette('crest_r', as_cmap=True)(1-(mc - mcmin)/(mcmax - mcmin))

    model_master = df_models_master.loc[constant]
    model_time_shift = df_models_time_shift.loc[constant]

    # Show report
    if show_report:
        display(df_log.loc[constant])
    else:
        print('')

    # Collect samples
    samples, component = extract_samples(df_meta, constant)
    mcc_mean = df_meta.query(f'component == "{component}"').groupby(['constant', 'rh'])['mc'].agg(['mean', 'std'])

    # Iterate over all samples
    time_exp, time_mapped_exp = [], []
    exp_data_master = []
    for sample in samples:
        # Initialize values
        ln_a = df_ln_a.loc[(constant, sample), 'ln_a']
        mc = df_ln_a.loc[(constant, sample), 'mc']
        rh = df_meta.loc[(sample, component), 'rh']
        mc_mean = mcc_mean.loc[(constant, rh), 'mean']
        color = mc2color(mc)
        
        # Collect experimental creep
        time, strain = collect_strains(sample, component, mode, df_creep)
        valid_idx = np.where( ~np.isclose(time, 0) )
        time = time[valid_idx]
        strain = strain[valid_idx]

        # Plot experimental creep
        axs[0].plot(time, strain, label=f'exp. ({mc_mean:.1f}%)', color=color, lw=1, ls='--')

        # Plot experimental creep in master time domain
        time_mapped = np.exp(ln_a) * time
        axs[1].scatter(np.log(time_mapped), strain, color=color, s=5, zorder=4)

        # Plot experimental time shifts
        axs[2].scatter([mc], [ln_a], color=color, zorder=4)

        # Store information
        time_exp.append(time)
        time_mapped_exp.append(time_mapped)
        exp_data_master.append(
            pd.DataFrame(
                data={'strain': strain},
                index=pd.MultiIndex.from_tuples([(sample, t) for t in np.log(time_mapped)], names=['sample', 'log(time)'])
            )
        )

    time_exp = np.concatenate(time_exp)
    time_mapped_exp = np.concatenate(time_mapped_exp)
    exp_data_master = pd.concat(exp_data_master)

    
    # Calculate master creep curve
    g = df_master.loc[constant, 'g'].to_numpy()
    tau = df_master.loc[constant, 'tau'].to_numpy()
    if settings['compliance_type'] == 'J_total':
        tau = tau[1:]
    time_mcc = np.geomspace(1, np.max(time_mapped_exp), 100)
    fun_mcc = lambda time: settings['fun_master_curve'](time, tau, g)
    strain_mcc = fun_mcc(time_mcc)
    time_mcc_model = model_master.index
    strain_mcc_model = model_master['fit']
    confidence_mcc = model_master['confidence']
    prediction_mcc = model_master['prediction']
    # confidence_mcc = model_master.eval_uncertainty(x=time_mcc, sigma=sigma)
    # prediction_mcc = model_master.dely_predicted

    # Plot master creep curve
    axs[1].plot(np.log(time_mcc), strain_mcc, color='k', lw=model_lw, zorder=5)
    if show_confidence:
        axs[1].fill_between(np.log(time_mcc_model), strain_mcc_model-confidence_mcc, strain_mcc_model+confidence_mcc, zorder=2, color=confidence_color, ec='none', alpha=0.5)
        axs[1].fill_between(np.log(time_mcc_model), strain_mcc_model-prediction_mcc, strain_mcc_model+prediction_mcc, zorder=1, color=confidence_color, ec='none', alpha=0.2)

    # Calculate time-shift curve
    p = df_time_shift.loc[constant, [c for c in df_time_shift.columns.str.lower() if 'p_' in c and not 'std' in c]].to_numpy()
    mc_ts = np.linspace(0, 25, 100)
    fun_ts = lambda mc: settings['fun_ln_a'](mc, p, df_ln_a.loc[constant, 'mc'].to_numpy(), df_ln_a.loc[constant, 'ln_a'].to_numpy())
    ln_a_ts = fun_ts(mc_ts)
    mc_ts_model = model_time_shift.index
    ln_a_ts_model = model_time_shift['fit']
    confidence_ts = model_time_shift['confidence']
    prediction_ts = model_time_shift['prediction']
    # confidence_ts = model_time_shift.eval_uncertainty(x=mc_ts, sigma=sigma)
    # prediction_ts = model_time_shift.dely_predicted

    # Plot time-shift function
    axs[2].plot(mc_ts, ln_a_ts, color='k', zorder=5)
    if show_confidence:
        axs[2].fill_between(mc_ts_model, ln_a_ts_model-confidence_ts, ln_a_ts_model+confidence_ts, zorder=2, color=confidence_color, ec='none', alpha=0.5)
        axs[2].fill_between(mc_ts_model, ln_a_ts_model-prediction_ts, ln_a_ts_model+prediction_ts, zorder=1, color=confidence_color, ec='none', alpha=0.2)
    

    # Plot selected moisture content
    ln_a = fun_ts(moisture)
    axs[2].scatter([moisture], [ln_a], marker='s', color=model_color, s=50, zorder=10)

    # Plot creep model at selected moisture content
    time_exp = np.geomspace(0.1, np.max(time_exp), 100)
    time_exp_mcc = np.exp(ln_a) * time_exp
    strain_exp = fun_mcc(time_exp_mcc)
    axs[0].plot(time_exp, strain_exp, label=f'model ({moisture:.1f}%)', color=model_color, lw=model_lw, zorder=10)

    # Show section of selected moisture content in master creep curve
    axs[1].plot(np.log(time_exp_mcc), strain_exp, color=model_color, lw=model_lw, zorder=10)
    

    # Plot creep model at experiment climates
    mc_exp_mean = df_meta.query(f'constant == "{constant}"').loc[samples].groupby('rh')['mc'].mean()
    for mc in sorted(mc_exp_mean):
        ln_a = fun_ts(mc)
        strain_exp = fun_mcc(np.exp(ln_a) * time_exp)
        axs[0].plot(time_exp, strain_exp, label=f'model ({mc:.1f}%)', color=mc2color(mc), lw=model_lw)


    # Format axes
    axs[1].sharey(axs[0])
    axs[1].tick_params(labelleft=False)
    [ax.grid(True) for ax in axs]

    # Select label
    match settings['compliance_type']:
        case 'J_total':
            ylabel = r'Compliance $J(t)$ [1/MPa]'
        case 'J_c':
            ylabel = r'Creep compliance $J_c(t)$ [1/MPa]'
        case 'J_relative':
            ylabel = r'Relative creep compliance $J_c(t) / J_0$ [-]'
        case _:
            raise ValueError(f"Invalid compliance type '{settings['compliance_type']}'.")

    # Add labels
    axs[0].set_title('Sample strains')
    axs[1].set_title('Master creep curve')
    axs[2].set_title('Time shifts')
    axs[0].set_xlabel('Experimental time [s]')
    axs[1].set_xlabel('Master curve time [ln(s)]')
    axs[2].set_xlabel('Moisture content [%]')
    axs[0].set_ylabel(ylabel)
    axs[2].set_ylabel('Time shift ln(a)')

    # Add legend
    handles, labels = axs[0].get_legend_handles_labels()
    unique = dict(zip(labels, handles)) # remove duplicate legend labels
    n = len(unique) - 1
    order = [int(i/2) if i%2 == 0 else int((n/2)+i//2+1) for i in range(n) ] + [int(n/2)]
    values, keys = list(unique.values()), list(unique.keys())
    values, keys = zip(*[(values[i], keys[i]) for i in order])
    axs[1].legend(values, keys, loc='upper left', title='Data type (MC)', fontsize='small', title_fontsize='small')

    # Display parameters
    display(df_master.loc[constant])
    display(df_time_shift.loc[constant].rename('time_shift').to_frame().T)
    df_export_tmsp_master[0] = exp_data_master
    df_export_tmsp_ln_a[0] = df_ln_a.loc[constant]


# Export function
def export_csv_tmsp(button):
    if len(df_export_tmsp_master) > 0 and len(df_export_tmsp_ln_a) > 0:
        export_path_master = f'export_creep_master_{w_tmsp_component.value}.csv'
        export_path_ln_a = f'export_creep_time_shift_{w_tmsp_component.value}.csv'
        df_export_tmsp_master[0].to_csv(export_path_master)
        df_export_tmsp_ln_a[0].to_csv(export_path_ln_a)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with w_tmsp.children[-1]:
            w_tmsp.children[-1].clear_output()
            print(f'({timestamp}) Successfully saved data to {export_path_master} and {export_path_ln_a}.')
button_tmsp = widgets.Button(description='Export CSVs', button_style='primary', icon='save')
button_tmsp.on_click(export_csv_tmsp)
display(button_tmsp)


# Initialize figure
fig, axs = plt.subplots(1, 3, figsize=(10, 4), gridspec_kw=dict(left=0.12, right=0.98, top=0.94, wspace=0.25, width_ratios=[0.36, 0.36, 0.28]))

# Call function
constants = [
    'E_cRR', 'E_tRR', 'E_avgRR', 'E_cTT', 'E_tTT', 'E_avgTT', 'E_cLL', 'E_tLL', 'E_avgLL',
    'G_RT', 'G_TR', 'G_avgRT', 'G_RL', 'G_LR', 'G_avgRL', 'G_TL', 'G_LT', 'G_avgTL',
    'E_cRT', 'E_tRT', 'E_avgRT', 'E_cRL', 'E_tRL', 'E_avgRL', 'E_cTL', 'E_tTL', 'E_avgTL'
]
w_tmsp_component = widgets.Dropdown(options=constants, description='component')
w_tmsp = widgets.interactive(
    visualize_tmsp_model,
    constant=w_tmsp_component,
    moisture=widgets.FloatSlider(value=12.5, min=0, max=25, step=0.1),
    show_report=False,
    show_confidence=False,
    df_master=widgets.fixed(df_master),
    df_time_shift=widgets.fixed(df_time_shift),
    df_ln_a=widgets.fixed(df_ln_a),
    df_log=widgets.fixed(df_log),
    df_models_master=widgets.fixed(df_models_master),
    df_models_time_shift=widgets.fixed(df_models_time_shift),
    df_meta=widgets.fixed(df_meta),
    df_creep=widgets.fixed(df_creep),
    axs=widgets.fixed(axs)
)
display(w_tmsp)

## Data export
---
Export the mean coefficients and their standard deviations.

In [ ]:
# Define data path
export_path = input_folder / 'viscoelasticity_coefficients.csv'

# Assemble data
df_export = df.drop(columns=['tau']).groupby(['rh', 'type', 'tau'])[['g', 'C_inv', 'mc', 'R2']].agg(['mean', 'std'])
df_export = df_export.loc[ [not ('cL_' in type_name or 'tL_' in type_name) for type_name in df_export.index.get_level_values('type')] ]
df_export.index = pd.MultiIndex.from_tuples( [(i[0], i[1].replace('x_', '_'),) + i[2:] for i in df_export.index], names=df_export.index.names )

taus = df_export.index.get_level_values('tau').unique()
series_g, series_g_std, series_tau = [], [], []
for i, tau in enumerate(taus):
    df_tau = df_export.query(f'tau == {tau}')
    data = df_tau.droplevel('tau')
    series_g.append( pd.Series(data=data[('g', 'mean')], name=f'g_{i}') )
    series_g_std.append( pd.Series(data=data[('g', 'std')], name=f'Std_g_{i}') )
    series_tau.append( pd.Series(data=df_tau.index.get_level_values('tau'), index=data.index, name=f'tau_{i}') )

series = series_tau + series_g + series_g_std
# series.append(pd.Series(data=df_export.groupby(['rh', 'type']).mean()[('mc', 'mean')], name='mc'))
series.append(pd.Series(data=df_export.groupby(['rh', 'type']).mean()[('R2', 'mean')], name='R2'))

df_export_pretty = pd.concat(series, axis=1)

# Save data
df_export_pretty.to_csv(export_path)
print(f'Successfully saved file {export_path}.')

Export the results of the clustering.

In [ ]:
# Define data path
clust_export_path = input_folder / 'cluster.csv'

# Save data
clust_df.to_csv(clust_export_path)
print(f'Successfully saved file {clust_export_path}.')

## References
---

<ul>
    <li><a href='https://doi.org/10.1007/s11043-023-09604-0'>Bengtsson, R., Bergeron, L., Afshar, R., Mousavi, M., Gamstedt, E.K., 2024. Evaluating the viscoelastic shear properties of clear wood via off-axis compression testing and digital-image correlation. Mech Time-Depend Mater 28, 2069–2083.</a></li>
    <li><a href='https://doi.org/10.1007/BF02472963'>Hayashi, K., Felix, B., Le Govic, C., 1993. Wood viscoelastic compliance determination with special attention to measurement problems. Mater. Struct. 26, 370–376.</a></li>
</ul>